In [2]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv("../EEG_Data.csv")

In [12]:
X = data.iloc[:, :4]
Y = data.iloc[:, 4:]

In [77]:
X_train = []
for x in range(0, len(X["eeg1"])):
    X_train.append([X["eeg1"][x], X["eeg3"][x], X["eeg4"][x]])
X_train = np.array(X_train, dtype='float')

X_train

array([[-0.08119012, -1.06034029, -0.05326613],
       [-0.09418102, -1.05526507, -0.07929981],
       [-0.10742053, -1.03184092, -0.08800521],
       ...,
       [ 0.03256479, -0.6228168 ,  0.07464068],
       [ 0.02238401, -0.62      ,  0.06881502],
       [-0.0067838 , -0.6125872 ,  0.05700178]])

In [78]:
Y_train = []
for y in range(0, len(Y["focused"])):
    Y_train.append([Y["focused"][y]])
Y_train = np.array(Y_train, dtype='float')

Y_train

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [84]:
class EEG_Model(nn.Module):
    def __init__(self):
        super(EEG_Model, self).__init__()
        self.fc1 = nn.Linear(3, 15)
        self.fc2 = nn.Linear(15, 15)
        self.fc3 = nn.Linear(15, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        # add layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.sigmoid(self.fc3(x))        
        return x

# initialize the NN
model = EEG_Model()
model = model.double()

print(model)

EEG_Model(
  (fc1): Linear(in_features=3, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=15, bias=True)
  (fc3): Linear(in_features=15, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [85]:
# specify loss function
criterion = nn.MSELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [90]:
n_epochs = 100

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for i in range(len(X_train)):
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        x = X_train[i]
        x = torch.from_numpy(x)
        
        y = Y_train[i]
        y = torch.from_numpy(y)

        outputs = model(x)
        
        # calculate the loss
        loss = criterion(outputs, y)
        
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # perform a single optimization step (parameter update)
        optimizer.step()
        
        # update running training loss
        train_loss += loss.item()
            
    # print avg training statistics 
    train_loss = train_loss/len(X_train)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, 
        train_loss
        ))

Epoch: 1 	Training Loss: 0.012405
Epoch: 2 	Training Loss: 0.009148
Epoch: 3 	Training Loss: 0.009281
Epoch: 4 	Training Loss: 0.007980
Epoch: 5 	Training Loss: 0.006587
Epoch: 6 	Training Loss: 0.007991
Epoch: 7 	Training Loss: 0.009688
Epoch: 8 	Training Loss: 0.008765
Epoch: 9 	Training Loss: 0.008064
Epoch: 10 	Training Loss: 0.007687
Epoch: 11 	Training Loss: 0.007969
Epoch: 12 	Training Loss: 0.007026
Epoch: 13 	Training Loss: 0.008577
Epoch: 14 	Training Loss: 0.007444
Epoch: 15 	Training Loss: 0.006018
Epoch: 16 	Training Loss: 0.006144
Epoch: 17 	Training Loss: 0.006154
Epoch: 18 	Training Loss: 0.006612
Epoch: 19 	Training Loss: 0.006403
Epoch: 20 	Training Loss: 0.007061
Epoch: 21 	Training Loss: 0.006612
Epoch: 22 	Training Loss: 0.006924
Epoch: 23 	Training Loss: 0.007125
Epoch: 24 	Training Loss: 0.006600
Epoch: 25 	Training Loss: 0.006924
Epoch: 26 	Training Loss: 0.007003
Epoch: 27 	Training Loss: 0.006584
Epoch: 28 	Training Loss: 0.007054
Epoch: 29 	Training Loss: 0.0

In [93]:
torch.save(model.state_dict(), 'EEG_Model.tf')